## The Zoning Problem 
The aim of this notebook is to generate the optimal zoning by formulating a linear assignment problem and using google
OR-Toolsfor solving the optimization problem.
The agent behaviours are taken from the previous study. The occupation and env lattices are random for the test problem .

### The Test Problem 
There are five agents :[Blue,Green,Yellow,Red,Violet]
The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1]
Total number of voxels in a lattice = x 
Value lattice for the agents are =[Bv],[Gv],[Yv],[Rv],[Vv]

####  Aim 
: To find the best combination of Zones to achieve the maximum occupancy value
#### Steps

1. Generate the lattices , Agent behaviours, agents, 
2. Find the possible origin locations for the agents
3. Simulate occupancy behaviour and retrieve the cost of occupancy for each agent at each position
4. Generate the Cost matrix
5. Use the MIP Solver to optimise the combination to get the permutation matrix 


## Initilization

In [1]:
import os
import itertools
import sys
from math import factorial as fac
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
from itertools import combinations
import pickle

## Base Lattices

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('voxelized_envelope_6m_voxel_size.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)

In [3]:
flattened_lattice = avail_lattice_base.flatten()

In [4]:
np.argwhere(flattened_lattice==1).flatten()

array([  12,   13,   14,   15,   16,   17,   18,   19,   20,   23,   24,
         25,   26,   27,   28,   29,   30,   31,   56,   57,   58,   67,
         68,   69,   78,   79,   80,   89,   90,   91,  100,  101,  102,
        111,  112,  113,  133,  134,  135,  136,  137,  138,  139,  140,
        141,  144,  145,  146,  147,  148,  149,  150,  151,  152,  155,
        156,  157,  158,  159,  160,  161,  162,  163,  188,  189,  190,
        191,  192,  193,  194,  195,  196,  199,  200,  201,  202,  203,
        204,  205,  206,  207,  232,  233,  234,  243,  244,  245,  254,
        255,  256,  265,  266,  267,  276,  277,  278,  287,  288,  289,
        309,  310,  311,  312,  313,  314,  315,  316,  317,  320,  321,
        322,  323,  324,  325,  326,  327,  328,  331,  332,  333,  334,
        335,  336,  337,  338,  339,  364,  365,  366,  367,  368,  369,
        370,  371,  372,  375,  376,  377,  378,  379,  380,  381,  382,
        383,  408,  409,  410,  419,  420,  421,  4

## Env Lattices

In [5]:
Private_sector_Owned = (pickle.load( open( "Privately Owned Housing.p", "rb" ) ))
Social_Sector_Rental = (pickle.load( open( "Social Sector Rental Housing.p", "rb" ) ))
Free_Sector_rental = (pickle.load( open( "Free Sector Rental Housing.p", "rb" ) )) 
Restaurants_and_Cafes = (pickle.load( open( "Restaurants and Cafe.p", "rb" ) )) 
Offices = (pickle.load( open( "Offices.p", "rb" ) )) 
Retail = (pickle.load( open( "Retail Stores.p", "rb" ) )) 
Parking = (pickle.load( open( "Parking.p", "rb" ) )) 

## Stencils

In [6]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
#print(stencil_von_neumann)

In [7]:
# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

In [8]:
# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von) 
#print(stencil_squareness_von)

In [9]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)
#print(stencil_cuboid)

In [10]:
Stencil_Find_neighbours_cuboid = tg.create_stencil("moore", 1, 1)

## Deriving all possible agent Center points

In [11]:
def all_possible_origins(flat_lattice,number_of_points_needed):
    reshaped_lattice= flat_lattice.reshape(99,4,16)*1
    reshaped_lattice_aggregated_value =np.sum(reshaped_lattice,axis=2)
    array = reshaped_lattice_aggregated_value.flatten()
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    
    indices =list(range(6,flat_lattice.size,16))
    points_1D=[]
    for points in range(number_of_points_needed):
        index= np.where(ranks==points)
        points_1D.append(indices[int(index[0])] )  
    return points_1D


In [12]:
all_possible_origins(Private_sector_Owned,20)

[6326,
 2582,
 4598,
 4582,
 4518,
 2646,
 2662,
 2678,
 2694,
 2710,
 2726,
 2742,
 2758,
 4502,
 4486,
 4470,
 2822,
 2838,
 2854,
 2870]

In [13]:
## The number of voxels which each agent has to occupy There are five agents :[Blue,Green,Yellow,Red,Violet]
## The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1] 
mass_size = np.count_nonzero((avail_lattice==1)) 
P_H = 415 # Private Housing
S_H= 301 # Social Housing
F_H=403 # Free sector Housing
R_C= 19 # Restaurant and Cafe 
R= 73 # Retail stores
O=93 # Offices
P=51 # Parking


In [14]:
#Divide the available lattice into grids to generate the necessary locations for origins??

In [15]:
def ranking_origins(value_lattice,origin_number):
    neighbours_for_agents = avail_lattice_base.find_neighbours(Stencil_Find_neighbours_cuboid)
    selected_neighbour_agents = neighbours_for_agents[avail_lattice_base.flatten()==1]
    neighbour_vals_for_agent= value_lattice.flatten()[selected_neighbour_agents]
    neighbour_vals_aggregated_for_agent =np.sum(neighbour_vals_for_agent,axis=1)
    
    array = neighbour_vals_aggregated_for_agent
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    
    a_lattice = ranks.flatten()
    Co_ordinates = []
    for item in range(0,(origin_number**2),origin_number):
        Co_ordinates.append(np.where(a_lattice==item))
    return np.array(Co_ordinates).flatten()

In [16]:
Agent_1_origins = all_possible_origins(Private_sector_Owned,10)
Agent_2_origins = all_possible_origins(Social_Sector_Rental,10)
Agent_3_origins = all_possible_origins(Free_Sector_rental,10)
Agent_4_origins = all_possible_origins(Restaurants_and_Cafes,10)
Agent_5_origins = all_possible_origins(Offices,10)
Agent_6_origins = all_possible_origins(Retail,10)
Agent_7_origins = all_possible_origins(Parking,10)

In [17]:
All_agent_origins = np.concatenate((Agent_1_origins,Agent_2_origins,Agent_3_origins,Agent_4_origins,Agent_5_origins,Agent_6_origins,Agent_7_origins))

In [18]:
all_permutations= All_agent_origins.reshape(7, len(Agent_1_origins))

In [19]:
Agent_3_origins

[6326, 2710, 2726, 2742, 2758, 4806, 4790, 4774, 2822, 2838]

In [20]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of random/argmax occupancy on a 2d squarish stencil 
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
     
      # definition of random/argmax occupancy on a 3d cubish stencil
    def random_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
      
        
    def random_occupy_cubish_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_cuboid, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
    def argmax_occupy_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
         # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    def one_neighbour_occupy_squareness_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_squareness.append(neighs)

        neighs_flatten = np.array(env.neigh_squareness).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        #neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)


        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
   
        
        
    def one_neighbour_occupy_cubish_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = np.argwhere(neighs_availibility_full_lattice==1).flatten()
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_cubish.append(neighs)

        neighs_flatten = np.array(env.neigh_cubish).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
                
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        
        #neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)

       # print(neighs_unq_base_value) 
        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )       
            
        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
        print(selected_neigh_1D_id)
        print(free_neighs_full_lattice)

In [21]:
# Agent init class

def initialize_agents_random_origin (stencil,avail_lattice):
    #finding origin 
    agn_num = 1
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), agn_num)
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

def initialize_agents_fixed_origin (stencil,avail_lattice,origin):
    #finding origin 
    agn_origins = np.unravel_index(origin,avail_lattice.shape)
    select_id = origin
    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

In [22]:
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_cubish = []
        self.neigh_squareness = []
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def random_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()
    
    def random_occupy_cubish_von_neumann_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
    
    def argmax_occupy_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.argmax_occupy_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
      
    def one_neighbour_occupy_cubish_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_cubish_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        

In [23]:
Agent_P = agent(np.unravel_index(400,init_avail_lattice.shape),stencil_cuboid,400)
Agent_S = agent(np.unravel_index(1260,init_avail_lattice.shape),stencil_cuboid,1260)
Agent_F = agent(np.unravel_index(3492,init_avail_lattice.shape),stencil_cuboid,3492)
Agent_R = agent(np.unravel_index(5440,init_avail_lattice.shape),stencil_cuboid,5440)
Agent_O = agent(np.unravel_index(6085,init_avail_lattice.shape),stencil_cuboid,6085)
Agent_RE = agent(np.unravel_index(4361,init_avail_lattice.shape),stencil_cuboid,4361)
Agent_PR = agent(np.unravel_index(4557,init_avail_lattice.shape),stencil_cuboid,4557)


#6326,2582,4598,4582,4518,2646,2662,2678,2694,2710,2726,2742,2758,4502,4486,4470,2822,2838,2854,2870
env_P = {"availibility": avail_lattice,"enviornment": Private_sector_Owned}
env_S = {"availibility": avail_lattice,"enviornment": Social_Sector_Rental}
env_F = {"availibility": avail_lattice,"enviornment": Free_Sector_rental}
env_R = {"availibility": avail_lattice,"enviornment": Restaurants_and_Cafes}
env_O = {"availibility": avail_lattice,"enviornment": Offices}
env_RE = {"availibility": avail_lattice,"enviornment": Retail}
env_PR = {"availibility": avail_lattice,"enviornment": Parking}

    
env_1 = environment(env_P,Agent_P,413,"one_neighbour_occupy_cubish_behaviour")
env_2 = environment(env_S,Agent_S,301,"one_neighbour_occupy_cubish_behaviour")
env_3 = environment(env_F,Agent_F,403,"one_neighbour_occupy_cubish_behaviour")
env_4 = environment(env_R,Agent_R,19,"one_neighbour_occupy_cubish_behaviour")
env_5 = environment(env_O,Agent_O,73,"one_neighbour_occupy_cubish_behaviour")
env_6 = environment(env_RE,Agent_RE,93,"one_neighbour_occupy_cubish_behaviour")
env_7 = environment(env_PR,Agent_PR,51,"one_neighbour_occupy_cubish_behaviour")



In [24]:
np.unravel_index(4598,init_avail_lattice.shape)

(26, 2, 0)

In [25]:
Agent_P.origin

array([2, 4, 4])

In [26]:
env_availability_viz = []
env_list =[env_1,env_2,env_3,env_4,env_5,env_6,env_7]
number_steps = max(map(lambda e:e.number_of_iterations,env_list))
for a in range(number_steps):
    # print(env.availibility)
    #print(env.agent_origin)

    for e in env_list:
        if a < e.number_of_iterations :
            #print(a)
            #print(e.number_of_iterations)
            if e.method_name == "one_neighbour_occupy_squareness_moore":
                e.one_neighbour_occupy_squareness_moore()
                
            elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                e.one_neighbour_occupy_cubish_agents()
                
            elif e.method_name == "random_occupy_squareness_agents" :
                e.random_occupy_squareness_agents()
            
            elif e.method_name == "random_occupy_cubish_agents" :
                e.random_occupy_cubish_agents()  
                
            elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                e.random_occupy_cubish_von_neumann_agents()                           
                
            elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                e.one_neighbour_occupy_squareness_von_neumann()                
                
                
            elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                e.one_neighbour_occupy_squareness_behaviour()  
                
            elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                e.one_neighbour_occupy_cubish_behaviour()  
            
            elif e.method_name == "argmax_occupy_von_neumann" :
                e.argmax_occupy_von_neumann()  
                
            env_availability_viz.append(e.availibility-1)

586
[  12   13   14   15   16   17   18   19   20   23   24   25   26   27
   28   29   30   31   56   57   58   67   68   69   78   79   80   89
   90   91  100  101  102  111  112  113  133  134  135  136  137  138
  139  140  141  144  145  146  147  148  149  150  151  152  155  156
  157  158  159  160  161  162  163  188  189  190  191  192  193  194
  195  196  199  200  201  202  203  204  205  206  207  232  233  234
  243  244  245  254  255  256  265  266  267  276  277  278  287  288
  289  309  310  311  312  313  314  315  316  317  320  321  322  323
  324  325  326  327  328  331  332  333  334  335  336  337  338  339
  364  365  366  367  368  369  370  371  372  375  376  377  378  379
  380  381  382  383  408  409  410  419  420  421  430  431  432  441
  442  443  452  453  454  463  464  465  485  486  487  488  489  490
  491  492  493  496  497  498  499  500  501  502  503  504  507  508
  509  510  511  512  513  514  515  540  541  542  543  544  545  546
  

 6139 6140 6141 6142 6143 6144 6145 6146 6147]
1258
[  12   13   14   15   16   17   18   19   20   23   24   25   26   27
   28   29   30   31   56   57   58   67   68   69   78   79   80   89
   90   91  100  101  102  111  112  113  133  134  135  136  137  138
  139  140  141  144  145  146  147  148  149  150  151  152  155  156
  157  158  159  160  161  162  163  188  189  190  191  192  193  194
  195  196  199  200  201  202  203  204  205  206  207  232  233  243
  244  245  254  255  256  265  266  267  276  277  278  287  288  289
  309  310  311  312  313  314  315  316  317  320  321  322  323  324
  325  326  327  328  331  332  333  334  335  336  337  338  339  364
  365  366  367  368  369  370  371  372  375  376  377  378  379  380
  381  382  383  408  419  430  431  432  441  442  443  452  453  454
  463  464  465  485  486  487  488  489  490  491  492  493  496  497
  498  499  500  501  502  503  504  507  508  509  510  511  512  513
  514  515  540  541  542

 6142 6143 6144 6145 6146 6147]
5886
[  12   13   14   15   16   17   18   19   20   23   24   25   26   27
   28   29   30   31   56   57   58   67   68   69   78   79   80   89
   90   91  100  101  102  111  112  113  133  134  135  136  137  138
  139  140  141  144  145  146  147  148  149  150  151  152  155  156
  157  158  159  160  161  162  163  188  189  190  191  192  193  194
  195  196  199  200  201  202  203  204  205  206  207  232  243  254
  255  256  265  266  267  276  277  278  287  288  289  309  310  311
  312  313  314  315  316  317  320  321  322  323  324  325  326  327
  328  331  332  333  334  335  336  337  338  339  364  365  366  367
  368  369  370  371  372  375  376  377  378  379  380  381  382  383
  408  430  441  442  443  452  453  454  463  464  465  485  486  487
  488  489  490  491  492  493  496  497  498  499  500  501  502  503
  504  507  508  509  510  511  512  513  514  515  540  541  542  543
  544  545  546  547  548  551  552  553

4021
[  12   13   14   15   16   17   18   19   20   23   24   25   26   27
   28   29   30   31   56   57   58   67   68   69   78   79   80   89
   90   91  100  101  102  111  112  113  133  134  135  136  137  138
  139  140  141  144  145  146  147  148  149  150  151  152  155  156
  157  158  159  160  161  162  163  188  189  190  191  192  193  194
  195  196  199  200  201  202  203  204  205  206  207  232  254  265
  266  267  276  277  278  287  288  289  309  310  311  312  313  314
  315  316  317  320  321  322  323  324  325  326  327  328  331  332
  333  334  335  336  337  338  339  364  365  366  367  368  369  370
  371  372  375  376  377  378  379  380  381  382  383  441  443  452
  453  454  463  464  465  485  486  487  488  489  490  491  492  493
  496  497  498  499  500  501  502  503  504  507  508  509  510  511
  512  513  514  515  540  541  542  543  544  545  546  547  548  551
  552  553  554  555  556  557  558  559  617  618  619  628  629  630
 

90
[  12   13   14   15   16   17   18   19   20   23   24   25   26   27
   28   29   30   31   56   89   90   91  100  101  102  111  112  113
  133  134  135  136  137  138  139  140  141  144  145  146  147  148
  149  150  151  152  155  156  157  158  159  160  161  162  163  188
  189  190  191  192  193  194  195  196  199  200  201  202  203  204
  205  206  207  232  265  267  276  277  278  287  288  289  309  310
  311  312  313  314  315  316  317  320  321  322  323  324  325  326
  327  328  331  332  333  334  335  336  337  338  339  364  365  366
  367  368  369  370  371  372  375  376  377  378  379  380  381  382
  383  441  443  452  453  454  463  464  465  485  486  487  488  489
  490  491  492  493  496  497  498  499  500  501  502  503  504  507
  508  509  510  511  512  513  514  515  540  541  542  543  544  545
  546  547  548  551  552  553  554  555  556  557  558  559  617  618
  619  628  629  630  639  640  641  661  662  663  664  665  666  667
  6

277
[  12   13   14   15   16   17   18   19   20   23   24   25   26   27
   28   29   30   31  100  102  111  112  113  133  134  135  136  137
  138  139  140  141  144  145  146  147  148  149  150  151  152  155
  156  157  158  159  160  161  162  163  188  189  190  191  192  193
  194  195  196  199  200  201  202  203  204  205  206  207  276  277
  278  287  288  289  309  310  311  312  313  314  315  316  317  320
  321  322  323  324  325  326  327  328  331  332  333  334  335  336
  337  338  339  364  365  366  367  368  369  370  371  372  375  377
  378  379  380  381  382  383  452  453  454  463  464  465  485  486
  487  488  489  490  491  492  493  496  497  498  499  500  501  502
  503  504  507  508  509  510  511  512  513  514  515  543  544  545
  546  547  548  554  555  556  557  558  559  617  619  628  629  630
  639  640  641  661  662  663  664  665  666  667  668  669  672  673
  674  675  676  677  678  679  680  683  684  685  686  687  688  689
  

4006
[  12   13   14   15   16   17   18   19   20   23   24   25   26   27
   28   29   30   31  111  113  133  134  135  136  137  138  139  140
  141  144  145  146  147  148  149  150  151  152  155  156  157  158
  159  160  161  162  163  188  189  190  191  192  193  194  195  196
  199  200  201  202  203  204  205  206  207  287  288  289  309  310
  311  312  313  314  315  316  317  320  321  322  323  324  325  326
  327  328  331  332  333  334  335  336  337  338  339  366  367  368
  369  370  371  372  377  378  379  380  381  382  383  463  464  465
  485  486  487  488  489  490  491  492  493  496  497  498  499  500
  501  502  503  504  507  508  509  510  511  512  513  514  515  543
  544  545  546  547  548  554  555  556  557  558  559  628  630  639
  640  641  661  662  663  664  665  666  667  668  669  672  673  674
  675  676  677  678  679  680  683  684  685  686  687  688  689  690
  691  720  721  722  723  724  731  732  733  734  735  760  761  762
 

628
[  12   13   14   15   16   17   18   19   20   23   24   25   26   27
   28   29   30   31  133  134  135  136  137  138  139  140  141  144
  145  146  147  148  149  150  151  152  155  156  157  158  159  160
  161  162  163  188  189  190  191  192  193  194  195  196  199  200
  201  202  203  204  205  206  207  287  309  310  311  312  313  314
  315  316  317  320  321  322  323  324  325  326  327  328  331  332
  333  334  335  336  337  338  339  366  367  368  369  370  371  372
  377  378  379  380  381  382  383  463  485  486  487  488  489  490
  491  492  493  496  497  498  499  500  501  502  503  504  507  508
  509  510  511  512  513  514  515  543  544  545  546  547  548  554
  555  556  557  558  559  628  639  640  641  661  662  663  664  665
  666  667  668  669  672  673  674  675  676  677  678  679  680  683
  684  685  686  687  688  689  690  691  720  721  722  723  724  731
  732  733  734  735  760  762  793  804  805  806  815  816  817  837
  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
len(np.where(env_availability_viz[-1].flatten()==6325)[0])

0

In [28]:
p = pv.Plotter(notebook=True)

base_lattice = env_availability_viz[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = env_availability_viz[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = sum(map(lambda e:e.number_of_iterations,env_list))
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)


[6 6 6]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(168.72367820989257, 336.72367820989257, 303.72367820989257),
 (-105.0, 63.0, 30.0),
 (0.0, 0.0, 1.0)]